# Форматы Sparse матриц в SciPy

В машинном обучении матрицы часто состоят преимущественно из нулей (пример - матрица покупок пользователей). Такие матрицы называются разреженными (sparse).

Разреженные матрицы эффективно хранят только ненулевые элементы, что экономит память и время вычислений. В SciPy есть несколько форматов разреженных матриц в модуле `scipy.sparse`

На результат вычислений формат хранения не влияет - он определеят только то, как данные лежат "под капотом", НО<br>
1) может влиять на эффективность вычислений<br>
2) многие библиотеки требуют входные данные в каком-то определенном формате => полезно знать, какие бывают

## Распространённые форматы разреженных матриц в SciPy

| Формат | Название                 | Подходящие сценарии          | Где обеспечивается эффективность      |
|:--------|:--------------------------|:-------------------------------|:---------------------------|
| CSR    | Сжатая по строкам (Compressed Sparse Row) | Выбор строк, матрично-векторные операции | Операции по строкам       |
| CSC    | Сжатая по столбцам (Compressed Sparse Column) | Выбор столбцов, матрично-векторные операции | Операции по столбцам      |
| COO    | Координатный формат (Coordinate Format) | Простое создание              | Создание матриц           |
| LIL    | Список списков (List of Lists) | Пошаговое построение          | Назначение элементов      |
| DOK    | Словарь ключей (Dictionary of Keys) | Случайный доступ, назначение | Назначение элементов      |
| BSR    | Блочно-сжатая по строкам (Block Sparse Row) | Блочные матрицы              | Операции с блоками        |

На практике встречаются в основном первые три

---

## 1. Создание Sparse матрицы

In [ ]:
import numpy as np
from scipy import sparse

dense = np.array([
    [0, 0, 3],
    [4, 0, 0],
    [0, 5, 0]
])

# COO формат
coo = sparse.coo_matrix(dense)

# CSR формат
csr = sparse.csr_matrix(dense)

# CSC формат
csc = sparse.csc_matrix(dense)


---

## 2. COO (координатный) формат

COO хранит данные в виде тройки `(строка, столбец, значение)`.

In [ ]:
coo = sparse.coo_matrix(([3, 4, 5], ([0, 1, 2], [2, 0, 1])))
print(coo.toarray())

Легко создавать

Неэффективен для операций среза или арифметики.

---

## 3. CSR (сжатый по строкам) формат

Эффективен для **выбора строк** и умножения матрицы на вектор.

In [ ]:
csr = sparse.csr_matrix(dense)
print("Вторая строка:", csr[1].toarray())

Внутреннее устройство:
- `data`: список ненулевых значений
- `indices`: индексы столбцов
- `indptr`: указатели начала строк

---

## 4. CSC (сжатый по столбцам) формат

Сжимает **столбцы**; эффективен для доступа по столбцам.

In [ ]:
csc = sparse.csc_matrix(dense)
print("Второй столбец:", csc[:,1].toarray())

---

## 5. LIL (список списков)

Хорош для **пошагового построения** разреженной матрицы.

In [ ]:
lil = sparse.lil_matrix((3, 3))
lil[0, 2] = 3
lil[1, 0] = 4
lil[2, 1] = 5
print(lil.toarray())

Быстро добавлять элементы

Неэффективен для математических операций

---

## 6. DOK (словарь ключей)

Использует Python словарь `{(строка, столбец): значение}`.

In [ ]:
dok = sparse.dok_matrix((3, 3))
dok[0, 2] = 3
dok[1, 0] = 4
dok[2, 1] = 5
print(dok.toarray())

Отлично для случайного доступа и обновлений

Требует преобразования для быстрых операций

---

## 7. BSR (блочно-сжатый по строкам) формат

Полезен, когда матрица состоит из блоков.

In [ ]:
data = np.array([
    [[1, 0], [0, 1]],
    [[4, 5], [7, 8]]
])
bsr = sparse.bsr_matrix((data, [0, 2], [0, 1, 2]))
print(bsr.toarray())

---

## Преобразование между форматами

Преобразование между типами выполняется очень легко, есть для этого специальные методы

In [ ]:
csr = coo.tocsr()
csc = csr.tocsc()
lil = csr.tolil()


---

## Операции с разреженными матрицами

In [ ]:
A = sparse.csr_matrix([[1, 0, 2], [0, 3, 0]])
B = sparse.csr_matrix([[0, 4, 0], [0, 0, 5]])

# Умножение матрицы на вектор
v = np.array([1, 2, 3])
print(A.dot(v))

# Сложение
C = A + B
print(C.toarray())

# Транспонирование
print(A.T.toarray())